In [4]:
%load_ext autoreload
%autoreload 2
import syft as sy
sy.logger.remove()
import numpy as np
import pandas as pd
from syft.core.adp.entity import Entity
from syft.core.tensor.autodp.single_entity_phi import SingleEntityPhiTensor as SEPT
from syft.core.tensor.smpc.mpc_tensor import MPCTensor
import time
import uuid
from typing import Any
from typing import Dict
from syft.core.smpc.protocol.aby3 import ABY3
data = sy.Tensor(np.array([-2,0,-2,-2,0]))
import operator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
DOMAIN1_PORT = 8081
DOMAIN2_PORT = 8082
p1 =  sy.login(email="info@openmined.org",password="changethis",port=DOMAIN1_PORT)
p2 = sy.login(email="info@openmined.org",password="changethis",port=DOMAIN2_PORT)
t1 = data.send(p1)
t2 = data.send(p2)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into adp... done!

Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into musing_sutskever... done!


In [6]:
ROOT_DIR = "/home/azureuser/PySyft"
def load_data(csv_file: str) -> pd.DataFrame:
    return pd.read_csv(f"../trade_demo/datasets/{csv_file}")[0:10]

def get_user_details(unique_email: str) -> Dict[str, Any]:
    return {
        "name": "Sheldon Cooper",
        "email": unique_email,
        "password": "bazinga",
        "budget": 10,
    }

unique_email = f"{uuid.uuid4()}@caltech.edu"

# Canada
ca_root = sy.login(email="info@openmined.org", password="changethis", port=DOMAIN1_PORT)
ca_data = load_data(csv_file="ca - feb 2021.csv")


# canada_trade = (
#     (np.array(list(ca_data["Trade Value (US$)"])) / 1000)[0:10]
# ).astype(np.int64)
canada_trade = np.array([[1,2],[3,4]])
trade_partners = ((list(ca_data["Partner"])))[0:10]

entities = list()
for i in range(len(trade_partners)):
    entities.append(Entity(name=trade_partners[i]))

sampled_canada_dataset = sy.Tensor(canada_trade)
sampled_canada_dataset.public_shape = sampled_canada_dataset.shape
sampled_canada_dataset = sampled_canada_dataset.private(
      0, 3, entities=[entities[0].name]*canada_trade.shape[0],ndept=True
).tag("trade_flow")

# load dataset
ca_root.load_dataset(
    assets={"Canada Trade": sampled_canada_dataset},
    name="Canada Trade Data - First few rows",
    description=(
        "A collection of reports from Canada's statistics bureau about how "
        + "much it thinks it imports and exports from other countries."
    ),
    skip_checks=True,
    
)

assert len(ca_root.datasets) > 0



# Italy
it_root = sy.login(email="info@openmined.org", password="changethis", port=DOMAIN2_PORT)
it_data = load_data(csv_file="it - feb 2021.csv")

# italy_trade = ((np.array(list(it_data["Trade Value (US$)"])) / 1000)[0:10]).astype(
#     np.int64
# )
italy_trade = np.array([[1,2],[3,4]])
trade_partners = ((list(it_data["Partner"])))[0:10]

entities = list()
for i in range(len(trade_partners)):
    entities.append(Entity(name="Other Asia, nes"))

# Upload a private dataset to the Domain object, as the root owner
sampled_italy_dataset = sy.Tensor(italy_trade)
sampled_italy_dataset.public_shape = sampled_italy_dataset.shape
sampled_italy_dataset = sampled_italy_dataset.private(
    0, 3, entities=[entities[0].name]*canada_trade.shape[0],ndept=True,
).tag("trade_flow")

it_root.load_dataset(
    assets={"Italy Trade": sampled_italy_dataset},
    name="Italy Trade Data - First few rows",
    description=(
        "A collection of reports from iStat's statistics bureau about how "
        + "much it thinks it imports and exports from other countries."
    ),
    skip_checks=True,
    
)




Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into adp... done!


/tmp/ipykernel_108706/3138192200.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"../trade_demo/datasets/{csv_file}")[0:10]


Dataset is uploaded successfully !!! 🎉                                                                                                                                       

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!

Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into musing_sutskever... done!
Dataset is uploaded successfully !!! 🎉                                                                                                                                       

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!


/tmp/ipykernel_108706/3138192200.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"../trade_demo/datasets/{csv_file}")[0:10]


In [7]:
x = p1.datasets[-1]["Canada Trade"]
y = p2.datasets[-1]["Italy Trade"]

In [12]:
type(x)

syft.core.tensor.autodp.ndim_entity_phi.TensorWrappedNDimEntityPhiTensorPointer

In [14]:
x.entities

In [10]:
# working operations
ops = ["add","sub","mul","matmul","lt","gt","eq","ne","ge","le"]
idx = 2 # points to the op
op = getattr(operator,ops[idx])
res = op(x,y)
#For comparison increase the timeout
res.block_with_timeout(10)
val = res.get_copy()

Exception: Object with id <UID: 5ebaa1c30598c069a7d67a2d99779861> still doesn't exist after 10 second timeout.

In [28]:
val.round() == op(sampled_canada_dataset.decode(),sampled_italy_dataset.decode())

array([[ True,  True],
       [ True,  True]])

In [30]:
print(val.round()) #FPT result
print(op(sampled_canada_dataset.decode(),sampled_italy_dataset.decode())) #Acutal result

[[0. 0.]
 [0. 0.]]
[[False False]
 [False False]]


In [ ]:
# DEBUG CELL
# print((sampled_canada_dataset).decode())
# print((sampled_italy_dataset).decode())
# print(op(sampled_canada_dataset.decode(),sampled_italy_dataset.decode()).round())
# print(res.get_copy().round())

In [ ]:
p =res
public_result = p.publish(sigma=10000)

In [ ]:
pb_res = public_result.get_copy()

In [ ]:
pb_res

In [ ]:
#Below are debug cells
a=np.array([ -789925275 ,  798267317, -1708906764,  1806674886,  1165022425,   480477472,
    54663083  ,  78462192,  -974424744, -1538872148],dtype=np.int32)
b=np.array([  789925275,  -798266317 , 1708920764, -1806674886, -1165022425,  -480477472,
   -54663083  , -78459192  , 974424744 , 1538872148],dtype=np.int32)

In [ ]:
(a+b)/1000

In [ ]:
sampled_canada_dataset.decode()

In [ ]:
a=np.array([-1362805391  ,1471796679 , 2126726498   ,874113086 , -532526475, -1815214451,
  1602118491,  1561978677 , -970006488 ,  792178136],dtype=np.int32)

b=np.array([ 1362805391, -1470796679 ,-2084726498  ,-874113086  , 532526475  ,1815214451,
 -1602118491 ,-1558978677 ,  970006488  ,-792178136],dtype=np.int32)


In [ ]:
a= np.array([ 1300758168, -1917863768, -1667359440,  2060242384 ,  -49469704 ,-1556715208,
   -95689592 ,   94451448 , -656120896, -1904153536],dtype=np.int32)

b=np.array( [-1300758168 ,-1377103528  , 717686480, -2060242384,    49469704,  1556715208,
    95689592, -1389418744 ,  656120896  ,1904153536],dtype=np.int32)

In [ ]:
res.child[0].get_copy()+res.child[1].get_copy()

In [ ]:
def value(a):
    v1 = a.child[0].get_copy()
    v2 = a.child[1].get_copy()
    v = v1+v2
    return v.child